# Regression with Keras - A

## Introduction

Keras is a high-level API for building deep learning models. It has gained favor for its ease of use and syntactic simplicity facilitating fast development. Building a very complex deep learning network can be achieved with Keras with only few lines of code. In this notebook we perform Regression using Keras on a dataset containing compressive strength of different samples of concrete based on the volumes of the different materials that were used to make them.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3> 
1. <a href="#item1">Download and Clean Dataset</a>  
2. <a href="#item2">Import Keras</a>  
3. <a href="#item3">Build a Neural Network</a>  
4. <a href="#item4">Train and Test the Network</a>
5. <a href="#item4">Listing MSE</a>      
</font>
</div>

<a id="item1"></a>

## Download and Clean Dataset

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [19]:
import pandas as pd
import numpy as np

<strong>This dataset is about the compressive strength of different samples of concrete based on the volumes of the different materials that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's download the data and read it into a <em>pandas</em> dataframe.

In [20]:
concrete_data = pd.read_csv('https://ibm.box.com/shared/static/svl8tu7cmod6tizo6rk0ke4sbuhtpdfx.csv')
concrete_data.head()

Cement  Blast Furnace Slag  Fly Ash  Water  Superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   Coarse Aggregate  Fine Aggregate  Age  Strength  
0            1040.0           676.0   28     79.99  
1            1055.0           676.0   28     61.89  
2             932.0           594.0  270     40.27  
3             932.0           594.0  365     41.05  
4             978.4           825.5  360     44.30

So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [21]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [22]:
concrete_data.describe()

Cement  Blast Furnace Slag      Fly Ash        Water  \
count  1030.000000         1030.000000  1030.000000  1030.000000   
mean    281.167864           73.895825    54.188350   181.567282   
std     104.506364           86.279342    63.997004    21.354219   
min     102.000000            0.000000     0.000000   121.800000   
25%     192.375000            0.000000     0.000000   164.900000   
50%     272.900000           22.000000     0.000000   185.000000   
75%     350.000000          142.950000   118.300000   192.000000   
max     540.000000          359.400000   200.100000   247.000000   

       Superplasticizer  Coarse Aggregate  Fine Aggregate          Age  \
count       1030.000000       1030.000000     1030.000000  1030.000000   
mean           6.204660        972.918932      773.580485    45.662136   
std            5.973841         77.753954       80.175980    63.169912   
min            0.000000        801.000000      594.000000     1.000000   
25%            0.000000        932.000000      730.950000     7.000000   
50%            6.400000        968.000000      779.500000    28.000000   
75%           10.200000       1029.400000      824.000000    56.000000   
max           32.200000       1145.000000      992.600000   365.000000   

          Strength  
count  1030.000000  
mean     35.817961  
std      16.705742  
min       2.330000  
25%      23.710000  
50%      34.445000  
75%      46.135000  
max      82.600000

In [23]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

In [24]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Let's do a quick sanity check of the predictors and the target dataframes.

In [25]:
predictors.head()

Cement  Blast Furnace Slag  Fly Ash  Water  Superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   Coarse Aggregate  Fine Aggregate  Age  
0            1040.0           676.0   28  
1            1055.0           676.0   28  
2             932.0           594.0  270  
3             932.0           594.0  365  
4             978.4           825.5  360

In [26]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [27]:
n_cols = predictors.shape[1] # number of predictors

<a id="item1"></a>

## Import Keras

Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library.

#### Let's go ahead and import the Keras library

In [28]:
import keras

As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [29]:
from keras.models import Sequential
from keras.layers import Dense

## Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [30]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function creates a model that has one hidden layer with 10 neurons and a ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.

Let's import train_test_split function from scikit-learn in order to split the data into a training and test sets.

In [31]:
from sklearn.model_selection import train_test_split

Splitting the data by reserving 30% for test set.

In [32]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

## Train and Test the Network

In [33]:
# build the model
model = regression_model()

# fit the model
model.fit(predictors, target, epochs=50, verbose=2)

Epoch 1/50
 - 2s - loss: 162713.0784
Epoch 2/50
 - 1s - loss: 37224.5724
Epoch 3/50
 - 1s - loss: 7521.7508
Epoch 4/50
 - 1s - loss: 4022.7810
Epoch 5/50
 - 1s - loss: 3717.1204
Epoch 6/50
 - 1s - loss: 3529.3167
Epoch 7/50
 - 1s - loss: 3346.5502
Epoch 8/50
 - 1s - loss: 3163.6342
Epoch 9/50
 - 1s - loss: 2981.0822
Epoch 10/50
 - 1s - loss: 2804.7643
Epoch 11/50
 - 1s - loss: 2628.2677
Epoch 12/50
 - 1s - loss: 2462.3108
Epoch 13/50
 - 1s - loss: 2306.9324
Epoch 14/50
 - 1s - loss: 2155.0333
Epoch 15/50
 - 1s - loss: 2011.9373
Epoch 16/50
 - 1s - loss: 1870.3597
Epoch 17/50
 - 1s - loss: 1740.7970
Epoch 18/50
 - 1s - loss: 1628.1255
Epoch 19/50
 - 1s - loss: 1517.4085
Epoch 20/50
 - 1s - loss: 1423.8361
Epoch 21/50
 - 1s - loss: 1329.9434
Epoch 22/50
 - 1s - loss: 1247.5342
Epoch 23/50
 - 1s - loss: 1170.0549
Epoch 24/50
 - 1s - loss: 1100.5991
Epoch 25/50
 - 1s - loss: 1036.3380
Epoch 26/50
 - 1s - loss: 974.1541
Epoch 27/50
 - 1s - loss: 918.7405
Epoch 28/50
 - 1s - loss: 866.9891
E

We need to evaluate the model on test data.


In [34]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 1ms/step


282.3572304734906

Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Let's import the mean_squared_error function from Scikit-learn.

In [35]:
from sklearn.metrics import mean_squared_error
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

282.3572278071093 0.0


List of 50 mean squared errors with report of mean and the standard deviation of the mean squared errors.

In [36]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors without normalized data. \n Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 113.97527083609869
MSE 2: 121.85809249631023
MSE 3: 97.85232410616088
MSE 4: 93.37276303189472
MSE 5: 82.89159425408323
MSE 6: 68.05315544844446
MSE 8: 51.09919542479284
MSE 9: 53.2572838743142
MSE 10: 66.2245206863749
MSE 11: 51.68386670455192
MSE 12: 48.15723707452175
MSE 13: 54.43603451429448
MSE 14: 56.20001134286035
MSE 15: 46.763405710362306
MSE 16: 41.40380247665455
MSE 17: 45.33363515119336
MSE 18: 48.55772439407299
MSE 19: 55.9965196751468
MSE 20: 47.50610233047634
MSE 21: 45.826430212718385
MSE 22: 43.037698887698475
MSE 23: 42.700733345303334
MSE 24: 44.438770047283484
MSE 25: 47.825137993278624
MSE 26: 47.25139847542476
MSE 27: 46.4123166278728
MSE 28: 41.957469458718904
MSE 29: 60.82380977494817
MSE 30: 46.257858764006485
MSE 31: 57.61428828069693
MSE 32: 47.185911184761515
MSE 33: 45.965166357342866
MSE 34: 46.59094527161237
MSE 35: 55.74460406750923
MSE 36: 49.11185533097647
MSE 37: 48.24720184239755
MSE 38: 46.98823580695588
MSE 39: 56.70672279036933
MSE 40: 39.8